# Name - Parikshit Sahu
## Code alpha internship task 3
## Music generation with AI

In [32]:
import music21
import numpy as np

# Function to parse MIDI files and extract note sequences
def get_notes(midi_files):
    notes = []
    for file in midi_files:
        midi = music21.converter.parse(file)
        notes_to_parse = None
        parts = music21.instrument.partitionByInstrument(midi)
        
        if parts:  # if the file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
        
        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    
    return notes


In [34]:
from sklearn.preprocessing import LabelEncoder

def prepare_sequences(notes, sequence_length):
    # Get all unique pitches
    encoder = LabelEncoder()
    encoded_notes = encoder.fit_transform(notes)
    n_vocab = len(set(encoded_notes))

    # Create sequences for the LSTM
    network_input = []
    network_output = []

    for i in range(0, len(encoded_notes) - sequence_length, 1):
        seq_in = encoded_notes[i:i + sequence_length]
        seq_out = encoded_notes[i + sequence_length]
        network_input.append(seq_in)
        network_output.append(seq_out)

    # Reshape and normalize input data
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)

    # One-hot encode the output
    network_output = np.eye(n_vocab)[network_output]

    return network_input, network_output, encoder


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model


In [38]:
def train_model(model, network_input, network_output, epochs=100, batch_size=64):
    model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size)


In [40]:
def generate_notes(model, network_input, encoder, n_vocab, num_notes=500):
    # Choose a random starting point
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]

    prediction_output = []

    # Generate notes
    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = encoder.inverse_transform([index])[0]
        prediction_output.append(result)

        # Add the predicted note to the input sequence
        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    return prediction_output


In [42]:
from music21 import instrument, note, chord, stream

def create_midi(prediction_output, filename='output.mid'):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        # If the pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # If the pattern is a note
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase the offset to avoid stacking
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)


In [52]:
midi_files = [r'C:\Users\sahup\midi_files\101627.mid', r'C:\Users\sahup\midi_files\82625.mid',r'C:\Users\sahup\midi_files\112561.mid']  # List of MIDI files
notes = get_notes(midi_files)

sequence_length = 100
network_input, network_output, encoder = prepare_sequences(notes, sequence_length)
n_vocab = len(encoder.classes_)

model = build_model(network_input, n_vocab)
train_model(model, network_input, network_output, epochs=200, batch_size=128)

generated_notes = generate_notes(model, network_input, encoder, n_vocab, num_notes=500)
create_midi(generated_notes, 'generated_music.mid')


C:\Users\sahup\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - loss: 3.1518
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - loss: 3.3021
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 2.6242
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 2.7359
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - loss: 2.5942
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 2.4454
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - loss: 2.4157
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - loss: 2.4945
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - loss: 2.4164
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - loss: 2.3714
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 2.3955
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 2.3740
Epoch 13/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - loss: 2.3647
Epoch 14/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - loss: 2.4012
Epoch 15/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - loss: 2.3759
Epoch 16/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step 